# MatchSum Code
* pytorch를 이용하여 구현
* 핵심 : 좋은 요약은 좋지 않은 요약에 비해서 의미적으로 원본 text와 더 유사함
* 흐름 : 요약된 candidate text선정 - 원본 text와 candidate 사이의 유사도 계산 모델 생성 - margin-based triplet loss를 이용하여 학습

In [1]:
import torch
from torch import nn
from torch.nn import init

from transformers import BertModel, RobertaModel

C:\Users\pc\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


### Preprocess

get candidate

1. 각 text에서 sentence score가 높은(전체 text와 유사도가 큰) 문장을 5개 뽑아냄
2. 뽑아낸 문장 5개 중 2~3를 조합하여 candidate text를 형성

### MatchSum Model
* encoder로 BERT, RoBERTa를 사용

In [42]:
class MatchSum(nn.Module):
    
    # init정의, encoder 입력이 무엇이냐에 따라 BERT 또는 RoBERTa 사용 
    def __init__(self, candidate_num, encoder, hidden_size=768):
        super(MatchSum, self).__init__() #생성자 호출
        
        self.hidden_size = hidden_size
        self.candidate_num  = candidate_num
        
        if encoder == 'bert':
            self.encoder = BertModel.from_pretrained('bert-base-uncased')
        else:
            self.encoder = RobertaModel.from_pretrained('roberta-base')

    def forward(self, text_id, candidate_id, summary_id):
        # text_id는 원본 text 데이터
        # candidate_id는 후보 요약본 데이터
        # summary_id는 잘 요약된 데이터
        
        #배치 사이즈 결정
        batch_size = text_id.size(0)
        
        pad_id = 0     # for BERT, <pad> token 0으로, masking
        if text_id[0][0] == 0:
            pad_id = 1 # for RoBERTa, masking x??

        # get document embedding
        input_mask = ~(text_id == pad_id)
        out = self.encoder(text_id, attention_mask=input_mask)[0] # last layer
        #1 for tokens that are not masked,
        #0 for tokens that are masked.
        
        doc_emb = out[:, 0, :]
        assert doc_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size], doc_emb size확인
        
        # get summary embedding
        input_mask = ~(summary_id == pad_id)
        out = self.encoder(summary_id, attention_mask=input_mask)[0] # last layer
        summary_emb = out[:, 0, :]
        assert summary_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]

        # get summary score
        summary_score = torch.cosine_similarity(summary_emb, doc_emb, dim=-1) #실제 요약본과 원본 문서의 cosine similarity 계산

        # get candidate embedding
        candidate_num = candidate_id.size(1) #후보수를 candidate_id 사이즈로 지정
        candidate_id = candidate_id.view(-1, candidate_id.size(-1)) #candidate_id reshape
        input_mask = ~(candidate_id == pad_id)
        out = self.encoder(candidate_id, attention_mask=input_mask)[0]
        candidate_emb = out[:, 0, :].view(batch_size, candidate_num, self.hidden_size)  # [batch_size, candidate_num, hidden_size]
        assert candidate_emb.size() == (batch_size, candidate_num, self.hidden_size)
        
        # get candidate score
        doc_emb = doc_emb.unsqueeze(1).expand_as(candidate_emb)
        score = torch.cosine_similarity(candidate_emb, doc_emb, dim=-1) # [batch_size, candidate_num]
        assert score.size() == (batch_size, candidate_num)

        return {'score': score, 'summary_score': summary_score}

### Training

D : 원본 문서<br>
C : 후보 요약문<br>
C* : 정답 요약문<br>

L1 = max(0, f(D, C) - f(D, C\*) + r1)<br>
L2 = max(0, f(D, C_j) - f(D, C_i) + (j-i)*r2), (i < j)

margin-based triplet loss L = L1 + L2<br>
=> 학습

논문 저자 깃헙 : https://github.com/maszhongming/MatchSum